In [11]:
import pandas as pd
import numpy as np

In [12]:
df = pd.read_csv(r'C:\Users\Motasem-PC\Desktop\tuqa\data\Data-Metric Units.csv',encoding='utf-8')
df.head()

,Soil Type,Diameter (m),Period (Hours),Effective Stress (kPa),Q0 (kN),QSet (kN)
0,1.61,0.32512,1656.0,336.119417,1948.321061,3113.755120
1,1.62,0.32512,1656.0,320.318932,1872.701294,3674.231042
2,1.61,0.32512,1656.0,322.712945,1565.774003,3011.446023
3,1.63,0.32512,1680.0,284.408738,885.196098,3438.475297
4,1.61,0.32512,1656.0,333.246602,1877.149515,3389.544859


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Soil Type               104 non-null    float64
 1   Diameter (m)            104 non-null    float64
 2   Period (Hours)          104 non-null    float64
 3   Effective Stress (kPa)  104 non-null    float64
 4   Q0 (kN)                 104 non-null    float64
 5   QSet (kN)               104 non-null    float64
dtypes: float64(6)
memory usage: 5.0 KB


In [4]:
df.isna().sum()

Soil Type                 0
Diameter (m)              0
Period (Hours)            0
Effective Stress (kPa)    0
Q0 (kN)                   0
QSet (kN)                 0
dtype: int64

In [5]:
df.duplicated().sum()

0

In [ ]:
!pip install ydata-profiling

In [9]:
import ydata_profiling as yd

In [ ]:
yd.ProfileReport(df)

In [12]:
df.columns

Index(['Soil Type', 'Diameter (m)', 'Period (Hours)', 'Effective Stress (kPa)',
       'Q0 (kN)', 'QSet (kN)'],
      dtype='object')

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Separate features and target
X = df.drop('QSet (kN)', axis=1)
y = df['QSet (kN)']

# Min-Max Scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
print(f"X_train.shape is {X_train.shape}\nX_test.shape is {X_test.shape}\nX_train.shape is {y_train.shape}\nX_train.shape is {y_test.shape}")
print(X_test[:5], y_test[:6])

X_train.shape is (83, 5)
X_test.shape is (21, 5)
X_train.shape is (83,)
X_train.shape is (21,)
[[0.         0.         0.00810742 0.55486941 0.01801802]
 [1.         0.4        0.13225234 0.63354754 0.20810811]
 [1.         0.4        0.08158095 0.63354754 0.20810811]
 [1.         0.4        0.00557385 0.58975531 0.22972973]
 [0.         0.132      0.0600456  1.         0.0990991 ]] 30     978.608752
65    4021.192326
64    3429.578854
53    2402.039664
45    1636.945549
93    5511.346562
Name: QSet (kN), dtype: float64


In [17]:
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    r2 = r2_score(y_test, predictions)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    return r2, rmse


In [ ]:
!pip install catboost

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
# import xgboost as xgb
# from catboost import CatBoostRegressor
# import lightgbm as lgb

In [20]:
# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Support Vector Regressor': SVR(),
    'Decision Tree': DecisionTreeRegressor(),
    'K-Nearest Neighbors': KNeighborsRegressor(),
    'Random Forest': RandomForestRegressor(),
    'XGBoost': xgb.XGBRegressor(),
    'CatBoost': CatBoostRegressor(verbose=0),
    'LightGBM': lgb.LGBMRegressor()
}

# Train and evaluate models
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    r2, rmse = evaluate_model(model, X_test, y_test)
    results[name] = {'R²': r2, 'RMSE': rmse}

# Convert results to DataFrame for better visualization
results_df = pd.DataFrame(results).T
print(results_df)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000333 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 83
[LightGBM] [Info] Number of data points in the train set: 83, number of used features: 5
[LightGBM] [Info] Start training from score 4381.664413
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

In [21]:
results_df.sort_values(by='R²', ascending=False)

,R²,RMSE
CatBoost,0.893220,761.894066
Random Forest,0.889030,776.698677
LightGBM,0.872338,833.067576
XGBoost,0.869364,842.715786
K-Nearest Neighbors,0.820536,987.731322
Decision Tree,0.820192,988.677506
Linear Regression,0.805296,1028.814772
Support Vector Regressor,0.004240,2326.630935


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize Random Forest model
rf = RandomForestRegressor()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, scoring='r2', verbose=2)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_

# Train the best model
best_rf = grid_search.best_estimator_

r2, rmse = evaluate_model(best_rf, X_test, y_test)

# Display the results
results = {
    'Best Parameters': best_params,
    'R²': r2,
    'RMSE': rmse
}

results_df = pd.DataFrame([results])
print(results_df.sort_values(by='R²', ascending=False))


In [26]:
results_df['Best Parameters'][0]

{'bootstrap': True,
 'max_depth': 30,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 200}

In [27]:
df.columns

Index(['Soil Type', 'Diameter (m)', 'Period (Hours)', 'Effective Stress (kPa)',
       'Q0 (kN)', 'QSet (kN)'],
      dtype='object')

In [19]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
import joblib

# Assuming your data is in a DataFrame called 'df'
# Load your data
# df = pd.read_csv('your_data.csv')  # Uncomment and modify if you need to load your data from a file

# Define the features and target
features = ['Soil Type', 'Diameter (m)', 'Period (Hours)', 'Effective Stress (kPa)', 'Q0 (kN)']
target = 'QSet (kN)'  # Replace with your actual target column name

X = df[features]
y = df[target]

# Define the pipeline
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('rf', RandomForestRegressor(
        bootstrap=True,
        max_depth=30,
        min_samples_leaf=1,
        min_samples_split=2,
        n_estimators=200
    ))
])

# Train the model
pipeline.fit(X, y)

# Save the model
joblib_file = "rf_model.pkl"
joblib.dump(pipeline, joblib_file)

print(f"Model saved to {joblib_file}")


Model saved to rf_model.pkl


In [30]:
# Load the model
loaded_pipeline = joblib.load(joblib_file)
print(f"Model loaded from {joblib_file}")

# Predict on the test data
y_pred = loaded_pipeline.predict(X)

# Evaluate the model
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Model loaded from rf_model.pkl
Mean Squared Error: 94795.02477474378
R^2 Score: 0.9779759622009436
